In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
if 'init_modules' in globals():
    for m in [x for x in sys.modules.keys() if x not in init_modules]:
        del(sys.modules[m]) 
else:
    init_modules = sys.modules.keys()

from math import *
import numpy as np
import random
from matplotlib import pyplot as plt
from scipy.stats import ortho_group
from scipy.interpolate import griddata
import importlib

np.set_printoptions(precision=7)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from chemistry import *
from chemistry.utils import linalg
from chemistry.functions import GaussianWrapper
from chemistry.optimization import GradientDescent, delta_strategies, stop_strategies

importlib.reload(functions)
importlib.reload(optimization)
importlib.reload(stop_strategies)
importlib.reload(delta_strategies)

<module 'chemistry.optimization.delta_strategies' from '/home/george/work/np-complete/study/au/python_chemistry/chemistry/optimization/delta_strategies/__init__.py'>

In [6]:
molecule, struct = functions.Molecule.load_chemcraft_file('./work/C2H4.xyz')
# molecule, struct = functions.Molecule.load_chemcraft_file('./work/H2O.xyz')

path = optimization.optimize_structure_rfo(molecule, struct, 
                                           delta_strategies.RFO(0), 
                                           stop_strategies.GradNorm(1e-5))



new iteration
value = -78.1610489572241, grad norm = 1.8266706646442491e-06, delta norm = 2.3035651198913574e-06
delta norm = 2.3035651198913574e-06 [[  6.1804043e-07  -1.3032262e-06   8.7858793e-07   4.5314646e-07
   6.3201005e-07  -4.5884494e-07   5.1849031e-07  -3.5017410e-07
   7.4919385e-07   5.1046655e-07   6.2774372e-07   1.7726274e-07]]
break


In [7]:
eq = path[-1]
print(molecule.grad(eq))
normalized = utils.linalg.get_normal_coordinate(molecule, eq)

[  4.9045754e-07   7.1105719e-07   6.7127943e-07  -5.3047336e-07
  -2.9567308e-07  -2.5521107e-07   3.9728663e-08   2.0554815e-08
  -3.4550088e-07  -4.6223739e-07  -7.5920901e-07  -1.1139499e-07
   3.3076442e-07   6.9764197e-07   1.3838319e-07   1.3176011e-07
  -3.7437189e-07  -9.7555682e-08]


In [8]:
def sample_on_sphere(func, max_r, n_r, n_phi):
    vects = []
    values = []
    for axis_num in range(3):
        for r in np.linspace(0, max_r, n_r)[1:]:
            for phi in np.linspace(0, 2 * pi, n_phi):
                for sign in [-1, 1]:
                    x = r * cos(phi)
                    y = r * sin(phi)
                    z = sign * np.sqrt(abs(max_r**2 - x**2 - y**2))

                    vect = np.array([x, y, z])
                    vect = np.roll(vect, axis_num)
                    vects.append(vect)
                    values.append(normalized(vect))
        
    return vects, values

def plot_in_polar(phis, rs, vs, ax, n_grid=100, is_contour=False):
    max_r = np.max(rs)

    grid_phi = np.linspace(0.0, 2 * pi, n_grid)
    grid_r = np.linspace(0, 2 * pi, n_grid)
    grid_phi, grid_r = np.meshgrid(grid_phi, grid_r)

    data = griddata((phis, rs / np.max(rs) * 2 * pi), vs, (grid_phi, grid_r), method='cubic')
    grid_r = .5 * max_r * grid_r / pi

    if is_contour:
        ax.contour(grid_phi, grid_r, data, 200)
    else:
        ax.pcolormesh(grid_phi, grid_r, data)
    
def global_to_polar(vects, axis_num, orient, r_eps=1e-7):
    phis = []
    rs = []
    inds = []
    
    for i, vect in enumerate(vects):
        if orient * vect[axis_num] < 0:
            continue

        vect = np.array([vect[(axis_num + 1) % 3], vect[(axis_num + 2) % 3]])

        if abs(np.linalg.norm(vect)) < r_eps:
            continue

        phis.append(atan2(vect[1], vect[0]))
        if phis[-1] < 0:
            phis[-1] += 2 * pi
        rs.append(np.linalg.norm(vect))
        inds.append(i)
        
    return phis, rs, inds

    
def plot_all_spheres(vects, values, r_eps=1e-7, title=None, n_grid=100, is_contour=False, **fig_kw):
    fig, axs = plt.subplots(3, 2, subplot_kw={'projection': 'polar'}, **fig_kw)
    plt.suptitle(title)
    
    for axis_num in range(3):
        for orient_num, orient in enumerate([1, -1]):
            phis, rs, inds = global_to_polar(vects, axis_num, orient, r_eps)
            vs = [values[i] for i in inds]

            value = vs[np.argmin(rs)]
            for phi in set(phis):
                phis.append(phi)
                rs.append(0)
                vs.append(value)

            phis = np.array(phis)
            vs = np.array(vs)
            rs = np.array(rs)
            axs[axis_num][orient_num].set_title('{}'.format(orient * utils.linalg.eye(3, axis_num)))
            plot_in_polar(phis, rs, vs, axs[axis_num][orient_num], n_grid, is_contour)
            
    return axs

def plot_path(axs, path, c='r'):
    for axis_num, ax_row in enumerate(axs):
        for orient_num, ax in enumerate(ax_row):
            phis, rs, inds = global_to_polar(path, axis_num, 1 - orient_num * 2)
            if len(phis):
                ax.plot(phis, rs, c=c)
                ax.scatter(phis, rs)                
                if inds[-1] == len(path) - 1:
                    ax.scatter(phis[-1:], rs[-1:])

In [9]:
MAX_R = .05
N_PHI = 20
N_R = 20
vects, values = sample_on_sphere(molecule, MAX_R, N_R, N_PHI)
axs = plot_all_spheres(vects, values, figsize=(20, 20), n_grid=250, is_contour=True)

ValueError: shapes (18,12) and (3,) not aligned: 12 (dim 1) != 3 (dim 0)

In [10]:
MAX_R = .05

known_directions = []

for i in range(10):
    seed = random.randint(0, 2**30)
    np.random.seed(seed)
    print('RANDOM SEED = {}'.format(seed))

    #     dir = np.array([1, -1, .3])
    dir = np.random.randn(normalized.n_dims)
    dir = dir / np.linalg.norm(dir) * MAX_R
    path = optimization.optimize_on_sphere_rfo(normalized, MAX_R, dir, 
                                                         delta_strategies.RFO(0), 
                                                         stop_strategies.GradNorm(1e-8))

    dir = path[-1]
    flag = True
    for known_dir in known_directions:
        dot = known_dir.dot(dir) / np.linalg.norm(known_dir) / np.linalg.norm(dir)
        if dot > .99:
            print('KNOWN DIRECTION: angle = {}, dot = {}'.format(acos(dot), dot))
            flag = False
            break
        else:
            print('from previous: angle = {}, dot = {}'.format(acos(dot), dot))
    if flag:
        known_directions.append(dir)
        
print()
print('Found directions:')

for dir in known_directions:
    print(dir)     

RANDOM SEED = 120422823


new iteration
value = -78.15977128898601, grad norm = 8.883072860455855e-05, delta norm = 21.640095339341556
delta norm = 21.640095339341556
expected = -0.1023122500949588, real = -2.6387693282003966e-05, d = 0.999742086668434

delta norm = 10.820047669670778
expected = -0.025687069148335476, real = -7.513698992056561e-05, d = 0.9970749099678647

delta norm = 5.410023834835389
expected = -0.006476270599381758, real = 3.076494381559769e-05, d = 1.0047504104937393

delta norm = 2.7050119174176945
expected = -0.0016463193059943842, real = -5.645760225547747e-05, d = 0.9657067726473773

delta norm = 1.3525059587088473
expected = -0.00042520565457306833, real = -0.0001001376900262585, d = 0.7644958646497713

delta norm = 0.6762529793544236
expected = -0.00011311432768050322, real = -7.38448906645317e-05, d = 0.347165896851634

delta norm = 0.3381264896772118
expected = -3.168503893874388e-05, real = -2.8257528668973464e-05, d = 0.10817440610998642



new iteration




new iteration
value = -78.15985565469612, grad norm = 0.0001368497648908159, delta norm = 2.050596546876909
delta norm = 2.050596546876909
expected = -0.0001922131480131307, real = 3.256894893866047e-05, d = 1.169441837227678

delta norm = 1.0252982734384546
expected = -6.651775721093283e-05, real = -1.507734793904092e-06, d = 0.9773333489112846

delta norm = 0.5126491367192273
expected = -2.5861674406558283e-05, real = -1.598494863230826e-05, d = 0.38190588973409145

delta norm = 0.25632456835961365
expected = -1.1081536153552109e-05, real = -9.871030556496407e-06, d = 0.10923626294064674



new iteration
value = -78.15986552572656, grad norm = 0.00011405985414332908, delta norm = 0.6062716796271629
delta norm = 0.6062716796271629
expected = -3.307243288567976e-05, real = -1.1738250833559505e-05, d = 0.6450744680884325

delta norm = 0.30313583981358144
expected = -2.035982760336305e-05, real = -1.7236379250107348e-05, d = 0.15341231832138744



new iteration
value = -78.159882762106

delta norm = 2.535648972200207
expected = -0.0007777986052660615, real = 8.399945301107437e-05, d = 1.1079964047792819

delta norm = 1.2678244861001036
expected = -0.0002215336022631953, real = 5.854559177009833e-06, d = 1.0264274092832846

delta norm = 0.6339122430500518
expected = -6.89253760391388e-05, real = -3.6933200135536026e-05, d = 0.46415671182462953

delta norm = 0.3169561215250259
expected = -2.4002331746454687e-05, real = -2.05660567189625e-05, d = 0.1431642168682111



new iteration
value = -78.15985909780125, grad norm = 0.00015999123024788327, delta norm = 24.152105706498187
delta norm = 24.152105706498187
expected = -0.031123830281682807, real = 4.662970641788888e-05, d = 1.0014981994823862

delta norm = 12.076052853249093
expected = -0.007807589900337856, real = 6.215755554705993e-05, d = 1.0079611706481113

delta norm = 6.038026426624547
expected = -0.0019652136400430406, real = 0.0001256433027094772, d = 1.0639336610276762

delta norm = 3.0190132133122733
expected =

delta norm = 0.7930451713136708
expected = -5.256304023559859e-05, real = -1.0014370317890098e-05, d = 0.8094788605643132

delta norm = 0.3965225856568354
expected = -2.2670503571347857e-05, real = -1.6706662393062288e-05, d = 0.263066109648441



new iteration
value = -78.15988673267465, grad norm = 8.426832946423615e-05, delta norm = 0.4917420466853553
delta norm = 0.4917420466853553
expected = -1.838285621814074e-05, real = -1.150229735458197e-05, d = 0.37429215470710336

delta norm = 0.24587102334267766
expected = -1.1997350640758e-05, real = -1.1032806881416946e-05, d = 0.08039639652310052



new iteration
value = -78.15989776548147, grad norm = 3.9897940857204775e-05, delta norm = 0.1600872471706093
delta norm = 0.1600872471706093
expected = -2.8217641394426944e-06, real = -2.9231812277430436e-06, d = 0.03594102245568238



new iteration
value = -78.15990068866245, grad norm = 3.82347866855591e-06, delta norm = 0.023657311249195696
delta norm = 0.023657311249195696
expected = -4.

delta norm = 0.7407619594191762
expected = -4.6094893824643155e-05, real = -2.3714295636523275e-05, d = 0.4855331324389517

delta norm = 0.3703809797095881
expected = -1.5202587963174287e-05, real = -1.298843233143998e-05, d = 0.14564333632521817



new iteration
value = -78.15986305982037, grad norm = 0.00010790667628747428, delta norm = 0.872014436778154
delta norm = 0.872014436778154
expected = -5.152330153818543e-05, real = -1.3647179628151207e-05, d = 0.7351260648924666

delta norm = 0.436007218389077
expected = -2.7514724759633216e-05, real = -2.1546427660723566e-05, d = 0.21691284034451705



new iteration
value = -78.15988460624911, grad norm = 4.4999567153798474e-05, delta norm = 0.23775013554094784
delta norm = 0.23775013554094784
expected = -4.346334646736242e-06, real = -4.028504221764706e-06, d = 0.07312608227491217



new iteration
value = -78.15988863475327, grad norm = 7.146885090495633e-06, delta norm = 0.08938524531430651
delta norm = 0.08938524531430651
expected = -1

In [11]:
for i, dir in enumerate(known_directions):
    path = modules.shs.shs(normalized, dir, MAX_R, MAX_R)

#     axs = plot_all_spheres(vects, values, figsize=(50, 50), n_grid=1000, is_contour=True)
#     plot_path(axs, path)
#     plt.show()
    
    path = list(map(lambda struct: normalized.transform(struct), path))
    molecule.save_chemcraft_path('path_{}.xyz'.format(i), path)

Iteration 0: r = 0.05. Sphere optimization started:


new iteration
value = -78.15990422423681, grad norm = 1.622225559019559e-12, delta norm = 5.339714329074842e-09
Sphere optimization returned path of length 1
Iteration 1: r = 0.1. Sphere optimization started:


new iteration
value = -78.15684677234978, grad norm = 2.432717210874652e-06, delta norm = 0.0009687712588267251
delta norm = 0.0009687712588267251
expected = -1.1058291193796287e-09, real = -1.1058460813728743e-09, d = 1.533871097112028e-05



new iteration
value = -78.15684677345585, grad norm = 7.974239161541973e-10, delta norm = 2.9533741329136555e-07
Sphere optimization returned path of length 2
Iteration 2: r = 0.15000000000000002. Sphere optimization started:


new iteration
value = -78.15235021765565, grad norm = 9.841998960746109e-06, delta norm = 0.0011774471658122572
delta norm = 0.0011774471658122572
expected = -5.383652326247323e-09, real = -5.384066525948583e-09, d = 7.693656205114145e-05



new iteration
value =



new iteration
value = -78.0581089994197, grad norm = 1.367274222297967e-10, delta norm = 4.499689003182696e-10
Sphere optimization returned path of length 3
Iteration 16: r = 0.8500000000000002. Sphere optimization started:


new iteration
value = -78.0510890858856, grad norm = 0.0005158989732597941, delta norm = 0.000991104365873252
delta norm = 0.000991104365873252
expected = -2.3802139157687716e-07, real = -2.3802108728432358e-07, d = 1.2784252354854577e-06



new iteration
value = -78.05108932390657, grad norm = 3.148956113618941e-07, delta norm = 6.273749126803811e-07
delta norm = 6.273749126803811e-07
expected = -9.132495213530623e-14, real = -5.542233338928781e-13, d = 5.068695585755641



new iteration
value = -78.05108932390682, grad norm = 1.1677662224924744e-10, delta norm = 2.7947218926618425e-10
Sphere optimization returned path of length 3
Iteration 17: r = 0.9000000000000002. Sphere optimization started:


new iteration
value = -78.04426443220953, grad norm = 0.0005815



new iteration
value = -77.98760288010735, grad norm = 6.392791470104542e-07, delta norm = 6.086528446884183e-07
delta norm = 6.086528446884183e-07
expected = -1.6417491228565372e-13, real = -8.526512829121202e-14, d = 0.4806445935974906



new iteration
value = -77.98760288010715, grad norm = 3.7985057835544343e-10, delta norm = 7.196342768338092e-10
Sphere optimization returned path of length 3
Iteration 28: r = 1.4500000000000006. Sphere optimization started:


new iteration
value = -77.9830514718788, grad norm = 0.0015186663198945866, delta norm = 0.0009063461177046356
delta norm = 0.0009063461177046356
expected = -6.047222401657668e-07, real = -6.047964689059881e-07, d = 0.0001227484873071889



new iteration
value = -77.98305207667522, grad norm = 7.826565296246073e-07, delta norm = 6.999851590945003e-07
delta norm = 6.999851590945003e-07
expected = -2.2543998263548122e-13, real = -3.836930773104541e-13, d = 0.7019743916981033



new iteration
value = -77.98305207667572, grad no



new iteration
value = -78.11872381380634, grad norm = 0.00010123009011808112, delta norm = 0.0011229040550879393
delta norm = 0.0011229040550879393
expected = -5.419395317893048e-08, real = -5.4189911224966636e-08, d = 7.458311724369364e-05



new iteration
value = -78.11872386799575, grad norm = 3.412288135246767e-08, delta norm = 3.529569314913546e-07
Sphere optimization returned path of length 2
Iteration 8: r = 0.44999999999999996. Sphere optimization started:


new iteration
value = -78.11104059317675, grad norm = 0.00012901893755498772, delta norm = 0.0010975890051559227
delta norm = 0.0010975890051559227
expected = -6.760865790446286e-08, real = -6.760397752714198e-08, d = 6.922748455523217e-05



new iteration
value = -78.11104066078079, grad norm = 4.5647409844921565e-08, delta norm = 3.756702476504783e-07
Sphere optimization returned path of length 2
Iteration 9: r = 0.49999999999999994. Sphere optimization started:


new iteration
value = -78.10328881475716, grad norm = 0.



new iteration
value = -78.02504018905094, grad norm = 0.000796050177889, delta norm = 0.000937844380307809
delta norm = 0.000937844380307809
expected = -3.423897567369419e-07, real = -3.4242164304032485e-07, d = 9.312867209239256e-05



new iteration
value = -78.02504053147264, grad norm = 4.797305265324172e-07, delta norm = 5.575998776905342e-07
delta norm = 5.575998776905342e-07
expected = -1.1957823600236388e-13, real = 4.263256414560601e-14, d = 1.3565244443375402



new iteration
value = -78.0250405314725, grad norm = 5.275093090914391e-11, delta norm = 6.366483772854927e-11
Sphere optimization returned path of length 3
Iteration 21: r = 1.1000000000000003. Sphere optimization started:


new iteration
value = -78.01906155110535, grad norm = 0.0008738341181498099, delta norm = 0.0009280997023915927
delta norm = 0.0009280997023915927
expected = -3.703858082221708e-07, real = -3.7041829159534245e-07, d = 8.770145197395024e-05



new iteration
value = -78.01906192152353, grad norm =

...

delta norm = 0.08687243319588361
expected = -0.0005323434918350061, real = -0.00045031857763433436, d = 0.1540826843170921



new iteration
value = -78.07374176678972, grad norm = 0.005306106583689485, delta norm = 0.03404426174897913
delta norm = 0.03404426174897913
expected = -3.5746353011949363e-05, real = -3.837112292615075e-05, d = 0.07342762808065958



new iteration
value = -78.07378013791273, grad norm = 0.000474489401467088, delta norm = 0.006773225829630451
delta norm = 0.006773225829630451
expected = -7.417893778266816e-07, real = -7.570537263745791e-07, d = 0.020577739455665243



new iteration
value = -78.07378089496659, grad norm = 1.0013053681929352e-05, delta norm = 0.00022883022653893086
delta norm = 0.00022883022653893086
expected = -7.726473244934877e-10, real = -7.727294359938242e-10, d = 0.00010627293686709387



new iteration
value = -78.07378089573918, grad norm = 1.1976417154999662e-08, delta norm = 2.589946079971943e-07
Sphere optimization returned path of leng

delta norm = 0.09423802330287959
expected = -0.0008359596168374, real = -0.0007344784296492435, d = 0.12139484389458889



new iteration
value = -78.0175342555086, grad norm = 0.012631809540023244, delta norm = 0.012568536554852822
delta norm = 0.012568536554852822
expected = -5.5244900172284235e-05, real = -5.718988231251387e-05, d = 0.03520654638100721



new iteration
value = -78.01759144539116, grad norm = 0.0007726477392542893, delta norm = 0.005637319549522211
delta norm = 0.005637319549522211
expected = -1.0502458760362211e-06, real = -1.1339761272211035e-06, d = 0.07972442748444049



new iteration
value = -78.01759257936746, grad norm = 0.0001059671654277472, delta norm = 0.0008604332365691514
delta norm = 0.0008604332365691514
expected = -1.951295472782653e-08, real = -1.9824824448733125e-08, d = 0.01598270099309218



new iteration
value = -78.01759259919214, grad norm = 2.420900885666567e-06, delta norm = 2.0583184725231035e-05
delta norm = 2.0583184725231035e-05
expected =

delta norm = 0.14821527380985713
expected = -0.0005267564156864261, real = -3.0114735238839785e-05, d = 0.9428298653000805

delta norm = 0.07410763690492857
expected = -0.00038940585022928856, real = -0.00037005169475889943, d = 0.04970175835569262



new iteration
value = -78.02936317449009, grad norm = 0.025998951504154426, delta norm = 0.061472963459733784
delta norm = 0.061472963459733784
expected = -0.00022702735901666092, real = -0.00025836090830466674, d = 0.13801662241820967



new iteration
value = -78.02962153539863, grad norm = 0.007730810485189345, delta norm = 0.01476680922994897
delta norm = 0.01476680922994897
expected = -1.899138459527959e-05, real = -1.9798983359464728e-05, d = 0.04252448051554242



new iteration
value = -78.0296413343816, grad norm = 0.0006061072973755306, delta norm = 0.0011462134600431057
delta norm = 0.0011462134600431057
expected = -1.0393491521941464e-07, real = -1.0400691508039017e-07, d = 0.0006927398826807567



new iteration
value = -78.0296

delta norm = 2.628055007053459e-07
expected = -8.022422829052361e-15, real = -5.542233338928781e-13, d = 68.08428360143479



new iteration
value = -78.03056433808192, grad norm = 1.3927103967464818e-10, delta norm = 5.224657461260113e-10
Sphere optimization returned path of length 5
Iteration 33: r = 1.7000000000000008. Sphere optimization started:


new iteration
value = -78.0289456123858, grad norm = 0.0902431179444803, delta norm = 0.06517155193728018
delta norm = 0.06517155193728018
expected = -0.0008873888379573025, real = -0.0008001273102706818, d = 0.09833516487258244



new iteration
value = -78.02974573969593, grad norm = 0.015379525870488549, delta norm = 0.00445755727490749
delta norm = 0.00445755727490749
expected = -2.2273101013727976e-05, real = -2.240615400239676e-05, d = 0.005973707414462758



new iteration
value = -78.02976814585006, grad norm = 0.00019281617418835382, delta norm = 0.00022611909554695397
delta norm = 0.00022611909554695397
expected = -2.9965215332179

GaussianException: 